In [1]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from toolz import compose
from numpy.linalg import norm
from sklearn.cluster import KMeans
from scipy.stats import linregress
from mpl_toolkits.mplot3d import Axes3D


import sys
sys.path.append('..')

import modules.plotting as pl
import modules.signals as sig
import modules.numpy_funcs as nf
import modules.pandas_funcs as pf
import modules.gait_metrics as gm
import modules.assign_sides as asi
import modules.sliding_window as sw
import modules.iterable_funcs as itf
import modules.linear_algebra as lin
import modules.phase_detection as pde

In [2]:
file_name = '2014-12-08_P004_Post_004'

In [3]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_best_pos = pd.read_pickle(best_pos_paths[0])

## Split trial into separate passes

In [4]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_best_pos = df_best_pos.applymap(pd.to_numeric)

# Cluster frames with k means to locate the 4 walking passes
frames = df_best_pos.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

# Sort labels so that the frames are in temporal order
labels = itf.map_to_whole(k_means.labels_)

# DataFrames for each walking pass in a trial
pass_dfs = nf.group_by_label(df_best_pos, labels)

In [5]:
df_pass = list(pass_dfs)[0]

In [6]:
frames = df_pass.index.values

In [7]:
_, direction_pass = gm.direction_of_pass(df_pass)

# Assign correct sides to feet
df_assigned = asi.assign_sides_pass(df_pass, direction_pass)

## Detect phases

In [8]:
signal_l = gm.foot_signal(df_assigned.L_FOOT, df_assigned.R_FOOT, direction_pass)

frames_interest = pde.frames_of_interest(signal_l)


In [9]:
df_phase_l = pde.foot_phases(frames_interest, direction_pass, df_assigned.L_FOOT)
df_phase_r = pde.foot_phases(frames_interest, direction_pass, df_assigned.R_FOOT)

## Calculate gait metrics

In [10]:
df_grouped_l = pde.group_stance_frames(df_phase_l, '_L')
df_grouped_r = pde.group_stance_frames(df_phase_r, '_R')


In [11]:
df_concat = pd.concat([df_grouped_l, df_grouped_r]).sort_values('frame').reset_index()
df_contact = pf.split_column(df_concat, column='index', delim='_', new_columns=['number', 'side'])

In [12]:
df_gait = gm.foot_contacts_to_gait(df_contact)

In [13]:
df_gait = pf.column_to_suffixes(df_gait, groupby_col='side', merge_col='number')

In [14]:
df_gait

,absolute_step_length_L,absolute_step_length_R,step_length_L,step_length_R,step_time_L,step_time_R,stride_length_L,stride_length_R,stride_time_L,stride_time_R,stride_velocity_L,stride_velocity_R,stride_width_L,stride_width_R
number,,,,,,,,,,,,,,
0,48.274148,40.126258,47.637915,39.081640,0.533333,0.400000,90.617329,86.491042,0.950000,0.933333,95.386662,92.668974,7.811684,9.096260
1,39.887085,36.417517,38.832927,34.682605,0.800000,0.583333,77.911480,72.992226,1.200000,1.383333,64.926233,52.765464,9.109515,11.106415
2,37.774028,NaN,35.316038,NaN,0.450000,NaN,69.177238,NaN,1.033333,NaN,66.945714,NaN,13.403532,NaN
